# 主站电量消耗分析

## 目标

分析主站（非Art）的电量消耗来源，识别消耗最多的 Bot 和用户。

## 两种计算方法

| 方法 | 原理 | 优点 | 适用场景 |
|------|------|------|----------|
| **直接差值法** | 总电量 - Art电量 | 简单快速 | 快速算总量 |
| **精确匹配法** | 通过时间匹配排除Art记录 | 可细分到Bot | 需要Bot级别分析 |

两种方法结果应基本一致（误差<5%），可互相验证。

## 参数

| 参数 | 说明 | 默认值 |
|------|------|--------|
| `days` | 查询天数 | 1（最近24小时） |
| `top_n` | Top Bot/User 数量 | 20 |

## 数据源

- `user_energy_bot_usage_logs` - 总电量消耗表（包含主站+Art）
- `art_task` - Art任务表（仅Art消耗）
- `bot` - Bot信息表
- `user` - 用户表
- `user_privy` - 用户邮箱信息

## Agent 运行模式

```bash
jq -r '.cells[] | select(.cell_type=="code") | .source | if type=="array" then join("") else . end' \
  main-site-energy-analysis.ipynb > temp/main-site-energy-analysis.ts

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/main-site-energy-analysis.ts 1              # 最近1天

deno run --no-check --allow-env --allow-net --allow-read --allow-write --allow-ffi \
  temp/main-site-energy-analysis.ts 7              # 最近7天
```

## Step 0: 环境配置

In [1]:
const MCP_HUB_URL = Deno.env.get("MCP_HUB_URL") || "http://52.12.230.109:3000/mcp";
const MCP_AUTH_TOKEN = (() => {
  let token = Deno.env.get("MCP_AUTH_TOKEN") || "";
  if (token && !token.startsWith("Bearer ")) token = `Bearer ${token}`;
  return token;
})();
const OUTPUT_DIR = "./reports";

let days = 1;
let topN = 20;
let mcpClient: any = null;

if (!MCP_AUTH_TOKEN) {
  console.error("❌ 请设置环境变量: export MCP_AUTH_TOKEN='your-token'");
} else {
  console.log("✅ 配置加载完成");
}

✅ 配置加载完成


## Step 1: 连接 MCP Hub

In [2]:
import { Client } from "npm:@modelcontextprotocol/sdk@1.24.3/client/index.js";
import { StreamableHTTPClientTransport } from "npm:@modelcontextprotocol/sdk@1.24.3/client/streamableHttp.js";

const transport = new StreamableHTTPClientTransport(new URL(MCP_HUB_URL), {
  requestInit: { headers: { Authorization: MCP_AUTH_TOKEN } },
});

mcpClient = new Client(
  { name: "main-site-energy-analysis", version: "1.0.0" },
  { capabilities: { tools: {}, sampling: {} } }
);

await mcpClient.connect(transport);
console.log("✅ Connected to MCP Hub");

✅ Connected to MCP Hub


## Step 2: 配置分析参数

支持两种模式：
- **CLI 模式**: 从命令行参数获取天数
- **Jupyter 模式**: 通过 prompt 交互输入

In [3]:
const args = typeof Deno !== "undefined" ? Deno.args : [];

if (args.length >= 1) {
  days = parseInt(args[0]) || 1;
  topN = parseInt(args[1]) || 20;
} else {
  const daysInput = prompt(`请输入查询天数（默认: 1）`, "1");
  days = parseInt(daysInput || "1") || 1;
  const topNInput = prompt(`请输入 Top N 数量（默认: 20）`, "20");
  topN = parseInt(topNInput || "20") || 20;
}

console.log(`✅ 分析参数: 最近 ${days} 天, Top ${topN}`);

✅ 分析参数: 最近 1 天, Top 20


## Step 3: 方法一 - 直接差值法

**原理**：主站电量 = 总电量 - Art电量

```
user_energy_bot_usage_logs.SUM(energy) - art_task.SUM(actual_energy_cost)
```

简单快速，适合快速验证总量。

In [4]:
console.log(`\n📊 方法一: 直接差值法`);
console.log(`${"-".repeat(50)}`);

// 查询总电量
const totalResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `SELECT COUNT(*) as cnt, COALESCE(SUM(energy), 0) as total_energy
          FROM my_shell_prod.user_energy_bot_usage_logs
          WHERE created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)`
  }
});
const totalData = JSON.parse((totalResult.content as any[])[0].text);
const totalLogsCount = totalData.data.rows[0]?.cnt || 0;
const totalLogsEnergy = totalData.data.rows[0]?.total_energy || 0;

// 查询 Art 电量
const artResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `SELECT COUNT(*) as cnt, COALESCE(SUM(actual_energy_cost), 0) as total_energy
          FROM my_shell_prod.art_task
          WHERE status = 'done' AND deleted_at IS NULL
            AND created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)`
  }
});
const artData = JSON.parse((artResult.content as any[])[0].text);
const artTaskCount = artData.data.rows[0]?.cnt || 0;
const artTaskEnergy = artData.data.rows[0]?.total_energy || 0;

// 计算主站电量
const mainSiteEnergyMethod1 = totalLogsEnergy - artTaskEnergy;
const mainSitePctMethod1 = totalLogsEnergy > 0 ? (mainSiteEnergyMethod1 / totalLogsEnergy * 100) : 0;
const artPctMethod1 = totalLogsEnergy > 0 ? (artTaskEnergy / totalLogsEnergy * 100) : 0;

console.log(`   总电量:   ${totalLogsEnergy.toLocaleString()} (${totalLogsCount.toLocaleString()} 条记录)`);
console.log(`   Art电量:  ${artTaskEnergy.toLocaleString()} (${artTaskCount.toLocaleString()} 条任务)`);
console.log(`   主站电量: ${mainSiteEnergyMethod1.toLocaleString()} (${mainSitePctMethod1.toFixed(1)}%)`);
console.log(`   Art占比:  ${artPctMethod1.toFixed(1)}%`);


📊 方法一: 直接差值法
--------------------------------------------------
   总电量:   100966 (30,924 条记录)
   Art电量:  62787 (18,985 条任务)
   主站电量: 38,179 (37.8%)
   Art占比:  62.2%


## Step 4: 方法二 - 精确匹配法

**原理**：通过时间匹配识别并排除 Art 记录

**匹配条件**：
- `user_id` + `bot_id` + `energy` 完全相等
- `usage_logs.created_date` 与 `art_task.updated_date` 相差 ±5秒内

**分类**：
- Art记录：能在 `art_task` 中找到匹配
- 主站记录：无法匹配的记录

In [5]:
console.log(`\n📊 方法二: 精确匹配法`);
console.log(`${"-".repeat(50)}`);

const matchResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- 精确匹配法：通过 art_task.updated_date 识别 Art 记录
          WITH art_done AS (
            SELECT user_id, bot_id, actual_energy_cost as energy, updated_date
            FROM my_shell_prod.art_task
            WHERE created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
              AND deleted_at IS NULL AND status = 'done'
          ),
          classified AS (
            SELECT u.user_id, u.bot_id, u.energy,
              CASE
                WHEN EXISTS (
                  SELECT 1 FROM art_done a
                  WHERE a.user_id = u.user_id AND a.bot_id = u.bot_id AND a.energy = u.energy
                    AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5
                ) THEN 'Art'
                ELSE 'Main Site'
              END as source
            FROM my_shell_prod.user_energy_bot_usage_logs u
            WHERE u.created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
          )
          SELECT source, COUNT(*) as records, SUM(energy) as total_energy,
            ROUND(SUM(energy) * 100.0 / SUM(SUM(energy)) OVER(), 2) as percentage
          FROM classified
          GROUP BY source`
  }
});

const matchData = JSON.parse((matchResult.content as any[])[0].text);
const sourceStats = matchData.data.rows || [];

let mainSiteEnergyMethod2 = 0, artEnergyMethod2 = 0;
let mainSiteRecords = 0, artRecords = 0;

for (const row of sourceStats) {
  if (row.source === 'Main Site') {
    mainSiteEnergyMethod2 = row.total_energy || 0;
    mainSiteRecords = row.records || 0;
  } else if (row.source === 'Art') {
    artEnergyMethod2 = row.total_energy || 0;
    artRecords = row.records || 0;
  }
}

const totalEnergyMethod2 = mainSiteEnergyMethod2 + artEnergyMethod2;
const mainSitePctMethod2 = totalEnergyMethod2 > 0 ? (mainSiteEnergyMethod2 / totalEnergyMethod2 * 100) : 0;
const artPctMethod2 = totalEnergyMethod2 > 0 ? (artEnergyMethod2 / totalEnergyMethod2 * 100) : 0;

console.log(`   主站: ${mainSiteEnergyMethod2.toLocaleString()} (${mainSitePctMethod2.toFixed(1)}%) - ${mainSiteRecords.toLocaleString()} 条`);
console.log(`   Art:  ${artEnergyMethod2.toLocaleString()} (${artPctMethod2.toFixed(1)}%) - ${artRecords.toLocaleString()} 条`);


📊 方法二: 精确匹配法
--------------------------------------------------
   主站: 40982 (0.0%) - 12,089 条
   Art:  59980 (0.0%) - 18,833 条


## Step 5: 两种方法对比验证

In [6]:
console.log(`\n📊 两种方法对比验证`);
console.log(`${"-".repeat(50)}`);

const diffEnergy = Math.abs(mainSiteEnergyMethod1 - mainSiteEnergyMethod2);
const diffPct = totalLogsEnergy > 0 ? (diffEnergy / totalLogsEnergy * 100) : 0;

console.log(`   方法一（差值法）主站电量: ${mainSiteEnergyMethod1.toLocaleString()}`);
console.log(`   方法二（匹配法）主站电量: ${mainSiteEnergyMethod2.toLocaleString()}`);
console.log(`   差异: ${diffEnergy.toLocaleString()} (${diffPct.toFixed(2)}%)`);

if (diffPct < 5) {
  console.log(`   ✅ 两种方法结果一致（误差 < 5%）`);
} else {
  console.log(`   ⚠️ 两种方法结果存在较大差异，请检查数据`);
}


📊 两种方法对比验证
--------------------------------------------------
   方法一（差值法）主站电量: 38,179
   方法二（匹配法）主站电量: 40982
   差异: 2,803 (2.78%)
   ✅ 两种方法结果一致（误差 < 5%）


## Step 6: 主站 Top Bots 分析

使用精确匹配法，获取主站消耗最多的 Bot。

In [7]:
console.log(`\n📊 主站 Top ${topN} Bots 电量消耗`);
console.log(`${"-".repeat(50)}`);

const topBotsResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- 主站 Top Bots（精确匹配法排除 Art）
          WITH art_done AS (
            SELECT user_id, bot_id, actual_energy_cost as energy, updated_date
            FROM my_shell_prod.art_task
            WHERE created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
              AND deleted_at IS NULL AND status = 'done'
          ),
          main_site AS (
            SELECT u.bot_id, u.user_id, u.energy
            FROM my_shell_prod.user_energy_bot_usage_logs u
            WHERE u.created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
              AND NOT EXISTS (
                SELECT 1 FROM art_done a
                WHERE a.user_id = u.user_id AND a.bot_id = u.bot_id AND a.energy = u.energy
                  AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5
              )
          )
          SELECT m.bot_id, b.name as bot_name, b.botType,
            SUM(m.energy) as total_energy,
            COUNT(*) as usage_count,
            COUNT(DISTINCT m.user_id) as unique_users,
            ROUND(SUM(m.energy) * 100.0 / (SELECT SUM(energy) FROM main_site), 2) as percentage
          FROM main_site m
          LEFT JOIN my_shell_prod.bot b ON m.bot_id = b.id
          GROUP BY m.bot_id, b.name, b.botType
          ORDER BY total_energy DESC
          LIMIT ${topN}`
  }
});

const topBotsData = JSON.parse((topBotsResult.content as any[])[0].text);
const topBots = topBotsData.data?.rows || [];

topBots.forEach((bot: any, i: number) => {
  const name = bot.bot_name || '(unknown)';
  const energy = Number(bot.total_energy).toLocaleString();
  const pct = bot.percentage || 0;
  const users = bot.unique_users || 0;
  console.log(`   ${(i + 1).toString().padStart(2)}. ${name.substring(0, 35).padEnd(35)} ${energy.padStart(10)} (${pct}%, ${users} 用户)`);
});


📊 主站 Top 20 Bots 电量消耗
--------------------------------------------------
    1. 🔞POV Missionary Insertion！🔞            6,624 (16.16%, 370 用户)
    2. 🔞Cum Facial AI Porn Maker💦👩           3,096 (7.55%, 499 用户)
    3. Wan POV Cowgirl Insertion                2,354 (5.74%, 555 用户)
    4. Rip Her Clothes V0.2                     2,210 (5.39%, 511 用户)
    5. Oral Sex-MAX                             2,044 (4.99%, 282 用户)
    6. Take off her clothes-PRO！!🎄🎄🔞         1,612 (3.93%, 335 用户)
    7. Porn videos🔞                            1,452 (3.54%, 50 用户)
    8. 🔞NSFW video maker (Adv: two prompt      1,261 (3.08%, 23 用户)
    9. Celebrity Porn V2                        1,203 (2.93%, 46 用户)
   10. Sexy 360-Degree Camera                   1,196 (2.92%, 6 用户)
   11. 🔞NSFW Video Playground🔞                  975 (2.38%, 44 用户)
   12. Sora 2 Photoreal Person                    910 (2.22%, 3 用户)
   13. 🔞Leg Fuck AI Porn Maker🦵💦               796 (1.94%, 59 用户)
   14. 🔞Missionary AI Porn Maker 🚀💦    

## Step 7: 主站 Top Users 分析

In [8]:
console.log(`\n📊 主站 Top ${topN} Users 电量消耗`);
console.log(`${"-".repeat(50)}`);

const topUsersResult = await mcpClient.callTool({
  name: "bytebase-execute_sql",
  arguments: {
    sql: `-- 主站 Top Users（精确匹配法排除 Art）
          WITH art_done AS (
            SELECT user_id, bot_id, actual_energy_cost as energy, updated_date
            FROM my_shell_prod.art_task
            WHERE created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
              AND deleted_at IS NULL AND status = 'done'
          ),
          main_site AS (
            SELECT u.bot_id, u.user_id, u.energy
            FROM my_shell_prod.user_energy_bot_usage_logs u
            WHERE u.created_date >= DATE_SUB(NOW(), INTERVAL ${days} DAY)
              AND NOT EXISTS (
                SELECT 1 FROM art_done a
                WHERE a.user_id = u.user_id AND a.bot_id = u.bot_id AND a.energy = u.energy
                  AND ABS(TIMESTAMPDIFF(SECOND, u.created_date, a.updated_date)) <= 5
              )
          )
          SELECT m.user_id, usr.source,
            COALESCE(NULLIF(up.email,''), NULLIF(up.google_email,''), NULLIF(up.apple_email,''), 'N/A') as email,
            SUM(m.energy) as total_energy,
            COUNT(*) as usage_count,
            COUNT(DISTINCT m.bot_id) as unique_bots,
            ROUND(SUM(m.energy) * 100.0 / (SELECT SUM(energy) FROM main_site), 2) as percentage
          FROM main_site m
          LEFT JOIN my_shell_prod.user usr ON m.user_id = usr.id
          LEFT JOIN my_shell_prod.user_privy up ON m.user_id = up.user_id
          GROUP BY m.user_id, usr.source, email
          ORDER BY total_energy DESC
          LIMIT ${topN}`
  }
});

const topUsersData = JSON.parse((topUsersResult.content as any[])[0].text);
const topUsers = topUsersData.data?.rows || [];

topUsers.forEach((user: any, i: number) => {
  const email = (user.email || 'N/A').substring(0, 30);
  const energy = Number(user.total_energy).toLocaleString();
  const pct = user.percentage || 0;
  const bots = user.unique_bots || 0;
  console.log(`   ${(i + 1).toString().padStart(2)}. ${email.padEnd(30)} ${energy.padStart(10)} (${pct}%, ${bots} bots)`);
});


📊 主站 Top 20 Users 电量消耗
--------------------------------------------------
    1. wjhlovesrj1314@gmail.com              637 (1.55%, 1 bots)
    2. john.t.frost41@gmail.com              570 (1.39%, 12 bots)
    3. dt5rkwf754@privaterelay.applei        546 (1.33%, 1 bots)
    4. n0462445@gmail.com                    483 (1.18%, 8 bots)
    5. j6044042@gmail.com                    441 (1.08%, 14 bots)
    6. rcnaywarburton@hotmail.com            436 (1.06%, 8 bots)
    7. ahorner327@gmail.com                  435 (1.06%, 12 bots)
    8. automatonx66@proton.me                405 (0.99%, 1 bots)
    9. plugnswitchbristol@outlook.com        392 (0.96%, 3 bots)
   10. mihku84@gmail.com                     384 (0.94%, 2 bots)
   11. skaterjuk@gmail.com                   366 (0.89%, 5 bots)
   12. lyingwei00@gmail.com                  365 (0.89%, 5 bots)
   13. unused-flocks.3j@icloud.com           359 (0.88%, 4 bots)
   14. m.rothbrust@hotmail.de                357 (0.87%, 12 bots)
   15. mits

## Step 8: 生成可视化图表

In [9]:
import * as Plot from "npm:@observablehq/plot";
import { document } from "jsr:@ry/jupyter-helper";

// 饼图数据：主站 vs Art
const pieData = [
  { category: "主站", value: mainSiteEnergyMethod2 },
  { category: "Art", value: artEnergyMethod2 }
];

if (Deno.args.length === 0) {
  // Jupyter 模式：显示饼图
  Plot.plot({
    document,
    title: `电量消耗分布（最近 ${days} 天）`,
    width: 500,
    height: 400,
    marks: [
      Plot.barY(pieData, { x: "category", y: "value", fill: "category", tip: true }),
      Plot.ruleY([0])
    ],
    color: { domain: ["主站", "Art"], range: ["#3B82F6", "#10B981"] }
  })
}

电量消耗分布（最近 1 天） 0 5,000 10,000 15,000 20,000 25,000 30,000 35,000 40,000 45,000 50,000 55,000 ↑ value Art 主站 category

In [10]:
// 柱状图：Top Bots
const botChartData = topBots.slice(0, 15).map((b: any) => ({
  bot_name: (b.bot_name || '(unknown)').substring(0, 25),
  total_energy: Number(b.total_energy)
}));

if (Deno.args.length === 0) {
  Plot.plot({
    document,
    title: `主站 Top 15 Bots 电量消耗`,
    width: 800,
    height: 500,
    marginLeft: 180,
    x: { label: "电量", grid: true },
    y: { label: null },
    marks: [
      Plot.barX(botChartData, {
        x: "total_energy",
        y: "bot_name",
        fill: "#3B82F6",
        sort: { y: "-x" },
        tip: true
      }),
      Plot.ruleX([0])
    ]
  })
}

主站 Top 15 Bots 电量消耗 🔞POV Missionary Insertio 🔞Cum Facial AI Porn Make Wan POV Cowgirl Insertion Rip Her Clothes V0.2 Oral Sex-MAX Take off her clothes-PRO！ Porn videos🔞 🔞NSFW video maker (Adv: Celebrity Porn V2 Sexy 360-Degree Camera 🔞NSFW Video Playground🔞 Sora 2 Photoreal Person 🔞Leg Fuck AI Porn Maker� 🔞Missionary AI Porn Make Glitch Lab⚡️ 0 1,000 2,000 3,000 4,000 5,000 6,000 电量 →

## Step 9: 导出报告

In [11]:
import pl from "npm:nodejs-polars";

try { await Deno.mkdir(OUTPUT_DIR, { recursive: true }); } catch {}

// 导出 Top Bots
const dfBots = pl.DataFrame({
  rank: topBots.map((_: any, i: number) => i + 1),
  bot_id: topBots.map((b: any) => b.bot_id),
  bot_name: topBots.map((b: any) => b.bot_name || ''),
  bot_type: topBots.map((b: any) => b.botType || ''),
  total_energy: topBots.map((b: any) => Number(b.total_energy)),
  usage_count: topBots.map((b: any) => Number(b.usage_count)),
  unique_users: topBots.map((b: any) => Number(b.unique_users)),
  percentage: topBots.map((b: any) => Number(b.percentage))
});

const botsFile = `${OUTPUT_DIR}/main-site-top-bots-${days}d.csv`;
dfBots.writeCSV(botsFile);
console.log(`💾 已保存: ${botsFile}`);

// 导出 Top Users
const dfUsers = pl.DataFrame({
  rank: topUsers.map((_: any, i: number) => i + 1),
  user_id: topUsers.map((u: any) => u.user_id),
  email: topUsers.map((u: any) => u.email || ''),
  source: topUsers.map((u: any) => u.source || ''),
  total_energy: topUsers.map((u: any) => Number(u.total_energy)),
  usage_count: topUsers.map((u: any) => Number(u.usage_count)),
  unique_bots: topUsers.map((u: any) => Number(u.unique_bots)),
  percentage: topUsers.map((u: any) => Number(u.percentage))
});

const usersFile = `${OUTPUT_DIR}/main-site-top-users-${days}d.csv`;
dfUsers.writeCSV(usersFile);
console.log(`💾 已保存: ${usersFile}`);

💾 已保存: ./reports/main-site-top-bots-1d.csv
💾 已保存: ./reports/main-site-top-users-1d.csv


## Step 10: 汇总报告

In [12]:
// 计算集中度指标
const top3BotEnergy = topBots.slice(0, 3).reduce((sum: number, b: any) => sum + Number(b.total_energy), 0);
const top10UserEnergy = topUsers.slice(0, 10).reduce((sum: number, u: any) => sum + Number(u.total_energy), 0);
const botConcentration = mainSiteEnergyMethod2 > 0 ? (top3BotEnergy / mainSiteEnergyMethod2 * 100) : 0;
const userConcentration = mainSiteEnergyMethod2 > 0 ? (top10UserEnergy / mainSiteEnergyMethod2 * 100) : 0;

console.log(`\n${'='.repeat(60)}`);
console.log(`📊 主站电量消耗分析报告（最近 ${days} 天）`);
console.log(`${'='.repeat(60)}`);
console.log(`\n【总体分布】`);
console.log(`   总电量:     ${totalLogsEnergy.toLocaleString()}`);
console.log(`   主站电量:   ${mainSiteEnergyMethod2.toLocaleString()} (${mainSitePctMethod2.toFixed(1)}%)`);
console.log(`   Art电量:    ${artEnergyMethod2.toLocaleString()} (${artPctMethod2.toFixed(1)}%)`);
console.log(`\n【集中度分析】`);
console.log(`   Top 3 Bot 占比:   ${botConcentration.toFixed(1)}%`);
console.log(`   Top 10 用户占比:  ${userConcentration.toFixed(1)}%`);
console.log(`\n【Top 3 Bots】`);
topBots.slice(0, 3).forEach((bot: any, i: number) => {
  console.log(`   ${i + 1}. ${bot.bot_name || '(unknown)'}: ${Number(bot.total_energy).toLocaleString()}`);
});
console.log(`${'='.repeat(60)}`);


📊 主站电量消耗分析报告（最近 1 天）

【总体分布】
   总电量:     100966
   主站电量:   40982 (0.0%)
   Art电量:    59980 (0.0%)

【集中度分析】
   Top 3 Bot 占比:   29.5%
   Top 10 用户占比:  11.5%

【Top 3 Bots】
   1. 🔞POV Missionary Insertion！🔞: 6,624
   2. 🔞Cum Facial AI Porn Maker💦👩: 3,096
   3. Wan POV Cowgirl Insertion: 2,354


In [13]:
// CLI 模式下退出进程
if (Deno.args.length > 0) Deno.exit(0);

---

## 附录

### 核心指标

| 指标 | 计算方式 | 目标 |
|------|----------|------|
| **主站占比** | 主站电量 / 总电量 × 100% | 监控变化趋势 |
| **Bot集中度** | Top 3 Bot电量 / 主站总电量 × 100% | 识别高消耗Bot |
| **用户集中度** | Top 10 用户电量 / 主站总电量 × 100% | 识别异常用户 |

### 注意事项

1. **时间窗口选择**：±5秒窗口经验证可准确匹配Art记录
2. **性能考虑**：EXISTS子查询在大数据量下可能较慢，建议限制时间范围（1-7天）
3. **数据一致性**：两种方法（差值法 vs 精确匹配法）结果应基本一致（误差<5%）

### 相关文档

- `main-site-revenue-attribution.ipynb` - 主站收入归因分析
- `bot-margin-analysis.md` - Art 站 Bot 毛利率分析
- `cost-trend-chart.md` - 每日成本趋势